# Author: Umang Srivastav

First Method: Taking 10% of the training set to train the model.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

model = keras.applications.resnet50.ResNet50(weights=None, input_shape=x_train.shape[1:], classes=10)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

num_samples_per_epoch = int(0.1 * x_train.shape[0])

for epoch in range(100):
    indices = np.random.choice(x_train.shape[0], size=num_samples_per_epoch, replace=False)
    x_train_subset, y_train_subset = x_train[indices], y_train[indices]

    model.fit(x_train_subset, y_train_subset, batch_size=32, epochs=1, verbose=1)

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
    print(f"Epoch {epoch+1}: Test accuracy = {test_acc:.4f}")

model.save("mod10per.pt")

157/157 [==============================] - 55s 43ms/step - loss: 2.7016 - accuracy: 0.2218
Epoch 1: Test accuracy = 0.1194
157/157 [==============================] - 6s 39ms/step - loss: 2.3515 - accuracy: 0.2736
Epoch 2: Test accuracy = 0.2388
157/157 [==============================] - 6s 40ms/step - loss: 2.1284 - accuracy: 0.3346
Epoch 3: Test accuracy = 0.2157
157/157 [==============================] - 6s 41ms/step - loss: 2.0105 - accuracy: 0.3776
Epoch 4: Test accuracy = 0.3488
157/157 [==============================] - 7s 42ms/step - loss: 2.2226 - accuracy: 0.3330
Epoch 5: Test accuracy = 0.3497
157/157 [==============================] - 6s 41ms/step - loss: 2.0909 - accuracy: 0.3610
Epoch 6: Test accuracy = 0.2606
157/157 [==============================] - 6s 41ms/step - loss: 2.0848 - accuracy: 0.3372
Epoch 7: Test accuracy = 0.3006
157/157 [==============================] - 6s 40ms/step - loss: 2.3827 - accuracy: 0.2668
Epoch 8: Test accuracy = 0.0960
157/157 [==============

In [ ]:
tf.keras.models.save_model(model, 'mod10per.h5')

Second Method: Using Transfer Learning (RN-50 on imagenet) with Data Augumentation.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

resnet = keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False, input_shape=x_train.shape[1:])

for layer in resnet.layers:
    layer.trainable = False

x = keras.layers.GlobalAveragePooling2D()(resnet.output)
x = keras.layers.Dense(10, activation='softmax')(x)
model = keras.models.Model(resnet.input, x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

num_samples = 1500


few_shot_indices = np.random.choice(x_train.shape[0], size=num_samples, replace=False)
x_train_few_shot, y_train_few_shot = x_train[few_shot_indices], y_train[few_shot_indices]

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='reflect')

history = model.fit(datagen.flow(x_train_few_shot, y_train_few_shot, batch_size=32), 
                    steps_per_epoch=len(x_train_few_shot) // 32, 
                    epochs=200, 
                    verbose=0)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Test accuracy = {test_acc:.4f}")


tf.keras.models.save_model(model, 'modDataAug.h5')

Test accuracy = 0.1054


Third Method: Using 50 samples per class of the train set to train the model.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torchvision.models import resnet50

num_classes = 10
num_samples_per_class = 50

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_indices = []
test_indices = []
for i in range(num_classes):
    indices = torch.arange(len(train_dataset))[torch.tensor(train_dataset.targets) == i]
    train_indices.extend(indices[:num_samples_per_class])
    test_indices.extend(indices[num_samples_per_class:num_samples_per_class+5])
train_subset = Subset(train_dataset, train_indices)
test_subset = Subset(test_dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=5, shuffle=False)

model = resnet50(pretrained=False, num_classes=num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

for epoch in range(100):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, 100, i+1, len(train_loader), loss.item()))

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Accuracy on test set: {:.2f}%'.format(accuracy))

torch.save(model.state_dict(), 'modelcl50.pt')

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/100], Step [10/50], Loss: 55.2695
Epoch [1/100], Step [20/50], Loss: 63.5718
Epoch [1/100], Step [30/50], Loss: 4.2903
Epoch [1/100], Step [40/50], Loss: 4.8622
Epoch [1/100], Step [50/50], Loss: 3.3820
Epoch [2/100], Step [10/50], Loss: 2.3510
Epoch [2/100], Step [20/50], Loss: 3.1156
Epoch [2/100], Step [30/50], Loss: 2.5117
Epoch [2/100], Step [40/50], Loss: 2.6587
Epoch [2/100], Step [50/50], Loss: 2.2569
Epoch [3/100], Step [10/50], Loss: 2.1706
Epoch [3/100], Step [20/50], Loss: 3.2530
Epoch [3/100], Step [30/50], Loss: 2.3451
Epoch [3/100], Step [40/50], Loss: 2.1797
Epoch [3/100], Step [50/50], Loss: 2.3652
Epoch [4/100], Step [10/50], Loss: 2.2100
Epoch [4/100], Step [20/50], Loss: 2.4540
Epoch [4/100], Step [30/50], Loss: 2.3355
Epoch [4/100], Step [40/50], Loss: 2.2884
Epoch [4/100], Step [50/50], Loss: 2.3245
Epoch [5/100], Step [10/50], Loss: 2.2336
Epoch [5/100], Step [20/50], Loss: 2.287